## Prepare Data

In [24]:
import pandas as pd
import json
import utils
import os
import pickle 
from tqdm import tqdm

In [16]:
config = utils.load_config("config.yaml")

In [1]:
movie_lines = open("datasets/movie_lines.txt", encoding="iso-8859-1").readlines()
movie_conv = open("datasets/movie_conversations.txt", encoding="iso-8859-1").readlines()

In [5]:
movie_conv[:3]

["u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L194', 'L195', 'L196', 'L197']\n",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L198', 'L199']\n",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L200', 'L201', 'L202', 'L203']\n"]

In [6]:
movie_lines[:3]

['L1045 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ They do not!\n',
 'L1044 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ They do to!\n',
 'L985 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I hope so.\n']

In [7]:
### splitting text using special lines
lines_dic = {}
for line in movie_lines:
    objects = line.split(" +++$+++ ")
    lines_dic[objects[0]] = objects[-1]

In [17]:
### generate question answer pairs
pairs = []
for con in movie_conv:
    ids = eval(con.split(" +++$+++ ")[-1])
    for i in range(len(ids)):
        qa_pairs = []
        
        if i == len(ids) - 1:
            break

        first = lines_dic[ids[i]].strip()  
        second = lines_dic[ids[i+1]].strip() 

        qa_pairs.append(' '.join(first.split()[:config["max_len"]]))
        qa_pairs.append(' '.join(second.split()[:config["max_len"]]))
        pairs.append(qa_pairs)

In [18]:
pairs[:5]

[['Can we make this quick? Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad. Again.',
  "Well, I thought we'd start with pronunciation, if that's okay with you."],
 ["Well, I thought we'd start with pronunciation, if that's okay with you.",
  'Not the hacking and gagging and spitting part. Please.'],
 ['Not the hacking and gagging and spitting part. Please.',
  "Okay... then how 'bout we try out some French cuisine. Saturday? Night?"],
 ["You're asking me out. That's so cute. What's your name again?",
  'Forget it.'],
 ["No, no, it's my fault -- we didn't have a proper introduction ---",
  'Cameron.']]

In [ ]:
pairs[220]

["I really, really, really wanna go, but I can't. Not unless my sister goes.",
 "I'm workin' on it. But she doesn't seem to be goin' for him."]

In [25]:
## First text of pair will act as input for second will be act as output
## So we will use the first one only to train the tokenizer.
## Save the data into multiple files each of having 10000 sample.
data_dir = "data/samples"
os.makedirs(data_dir,exist_ok=True)
text_data = []
file_count = 0

for sample in tqdm([x[0] for x in pairs]):
    text_data.append(sample)

    # once we hit the 10K mark, save to file
    if len(text_data) == 10000:
        with open(f'{data_dir}/sample_{file_count}.txt', 'w', encoding='utf-8') as fp:
            fp.write('\n'.join(text_data))
        text_data = []
        file_count += 1

100%|██████████| 221616/221616 [00:00<00:00, 1242944.52it/s]


In [31]:
## Save pairs data
pairs_df = pd.DataFrame(pairs, columns= ["first", "second"])
pairs_df.to_pickle("data/pairs.pkl", protocol=4)

In [32]:
## Remove datasets
!rm -rf datasets